In [24]:
import tensorflow as tf
import datetime
import os
import operator
import shutil

from tensorflow.keras import backend as K
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

import matplotlib.pyplot as plt
from PIL import Image

In [25]:
print(tf.__version__)
print(os.getcwd())

def create_folder(directory):
      if not os.path.exists(directory):
          os.makedirs(directory)
          return True
      return False

data_path = os.getcwd() + '/../data/'
alphabet_path = data_path + 'alphebet/'
create_folder(alphabet_path)

alphabet_list = os.listdir(data_path + 'ex/')
alphabet_list.sort()

img_size = 36

print(data_path)
print(alphabet_list)

2.12.0
/Users/naburang/Desktop/Metal/tensorflow_venv.nosync/code
/Users/naburang/Desktop/Metal/tensorflow_venv.nosync/code/../data/
['a.png', 'b.png', 'c.png', 'd.png', 'e.png', 'f.png', 'g.png', 'h.png', 'i.png', 'j.png', 'k.png', 'l.png', 'm.png', 'n.png', 'o.png', 'p.png', 'q.png', 'r.png', 's.png', 't.png', 'u.png', 'v.png', 'w.png', 'x.png', 'y.png', 'z.png', 'zz.png']


In [26]:
datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.05,
        zoom_range=0.01,
        fill_mode='constant',
        cval=255)


for j in alphabet_list:
    alpha = j.split('.')[0]
    if not create_folder(alphabet_path + alpha):
       break
    
    img = load_img(data_path + 'ex/' + j)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    
    if j == 'zz.png':
      for _ in range(10):
        print(_,j, alphabet_path + alpha)
        shutil.copy(data_path + 'ex/' + j, 
                    alphabet_path + alpha + '/' + alpha + str(_)+'.jpg')
    else:  
      i = 0
      for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=alphabet_path + alpha, 
                                save_prefix=alpha, 
                                save_format='jpg'):
          i += 1
          if i > 20:
              break

In [27]:
def prepare_data(size):
    datagen = ImageDataGenerator(rotation_range=10,
                                 shear_range=5,
                                 validation_split=0.3)
    train_generator = datagen.flow_from_directory(alphabet_path,
                                                  target_size=(size,size),
                                                  subset='training')
    val_generator = datagen.flow_from_directory(alphabet_path,
                                                target_size=(size,size),
                                                subset='validation')
    return train_generator, val_generator

In [28]:
def make_model(train,val, class_count, size = 36):
    K.clear_session()

    model_ckpt = ModelCheckpoint('ckpt.h5',save_best_only=True)
    reduce_lr = ReduceLROnPlateau(patience=8,verbose=1)
    early_stop = EarlyStopping(patience=20,verbose=2)
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    entry = L.Input(shape=(size,size,3))
    x = L.SeparableConv2D(64,(3,3),activation='relu',padding ='same')(entry)
    x = L.MaxPooling2D((2,2))(x)

    x = L.SeparableConv2D(128,(3,3),activation='relu',padding ='same')(x)
    x = L.MaxPooling2D((2,2))(x)

    x = L.SeparableConv2D(256,(2,2),activation='relu',padding ='same')(x)
    x = L.GlobalMaxPooling2D()(x)

    x = L.Dense(256)(x)
    x = L.LeakyReLU()(x)
    x = L.Dense(64,kernel_regularizer=l2(2e-4))(x)
    x = L.LeakyReLU()(x)
    x = L.Dense(class_count,activation='softmax')(x)

    model = Model(entry,x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    history = model.fit(train,
                        validation_data=val,
                        epochs=999,
                        callbacks=[model_ckpt,reduce_lr,early_stop],
                        verbose=1)
    return history

In [29]:
def show_losses(hist):
    _, loss_ax = plt.subplots(figsize=(10, 5))
    acc_ax = loss_ax.twinx()
    
    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
    
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')
    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

In [30]:
with tf.device('/GPU:0'):
    try:
        load_model('ckpt.h5')
    except:
        train_generator, val_generator = prepare_data(img_size)
        hist = make_model(train_generator, 
                          val_generator,
                          len(alphabet_list),
                          img_size)
        show_losses(hist)

In [31]:
model = load_model('ckpt.h5')

In [ ]:
call_num = 0

def devide_img(img_name):
    img = Image.open(data_path + img_name)
    width, height = img.size[0], img.size[1]
    length = int(width / height)

    area = (0 + call_num * height, 0,
            width / length * (call_num + 1), height)
    
    